# SMILES → fingerprints ECFP4 en format per ser input de MolForge
Transforma una taula amb **SMILES** en **fingerprints** amb el format necessati per fer-se servir com a input de MolForge.

**Entrada**: CSV amb columna `in_col_name`.

**Sortida**: CSV amb columnes `in_col_name` i `out_col_name_ECFP4`.

## Imports

In [1]:
# Per definir l'arrel del projecte
import os

# Pandas pels dataframes
import pandas as pd

# Per no mostrar els Warnings de RDKit
from rdkit import RDLogger
RDLogger.DisableLog("rdApp.*")

## Inputs (part a editar)

Arrel del projecte

In [2]:
os.chdir("/export/home/ddiestre/MolForge_Testing")

In [3]:
# Importem la funció smiles_to_fingerprint del nostre sourcecode que utilitza RDKit
from src.smiles_to_fp import smiles_to_fingerprint

Fingerprints en que transformar els SMILES

In [4]:
fp_type = "ECFP4"

Fitxer de fingerprints preprocessat (path a partir de MolForge_Testing/)

In [5]:
#input_path = "data/SMILES/MolForge_1.csv"
input_path = "data/SMILES/CoCoGraph_1.csv"

in_col_name = "SMILES_input"

Fitxer en que guardar l'output (path a partir de MolForge_Testing/)

In [6]:
#output_path = "data/MolForge_input/MolForge_MFinput_1.csv"
output_path = "data/MolForge_input/CoCoGraph_MFinput_1.csv"

out_col_name = "fingerprints_input_" + fp_type

## 1. Lectura del fitxer

In [7]:
# Lectura del fitxer
df = pd.read_csv(input_path, sep = ',', index_col = 0)
df.head(5)

,SMILES_input
id,
1,CCCCCN(CCCCc1c2cc3sc2nn13)C(=O)c1ccc(OCC)c(Cl)c1
2,O=C(Nc1n[nH]c2ccc(Br)cc12)N1CCCC1
3,CCN(C(=O)N(C)c1ccccc1OC)C(=Cc1ccc(OC)cc1)c1ccc...
4,CC1(Cl)OC(=O)C1(C)Cl
5,CCc1cc(C=CC(C)=Cc2cc(OC)ccc2C)ccc1C


## 2. Columna SMILES → Columna de fingerprints

In [8]:
fingerprints = []

# Apliquem el conversor SMILES -> fingerprint
fingerprints = df[in_col_name].apply(
    lambda s: smiles_to_fingerprint(s, fp_type=fp_type, n_bits=2048, return_bits=True)
)

# Guardem les fingerprints en el mateix format que l'input de MolForge
df[out_col_name] = fingerprints.apply(
    lambda lst: " ".join(str(x) for x in lst) if isinstance(lst, list) else lst
)

## 3. Guardar l'output

In [11]:
# Visualitzem el nou dataframe
df.head(5)

,SMILES_input,fingerprints_input_ECFP4
id,,
1,CCCCCN(CCCCc1c2cc3sc2nn13)C(=O)c1ccc(OCC)c(Cl)c1,70 80 85 94 162 237 294 347 366 378 412 425 51...
2,O=C(Nc1n[nH]c2ccc(Br)cc12)N1CCCC1,74 91 119 122 218 328 369 378 650 708 728 766 ...
3,CCN(C(=O)N(C)c1ccccc1OC)C(=Cc1ccc(OC)cc1)c1ccc...,25 54 80 249 252 294 322 376 431 650 694 695 7...
4,CC1(Cl)OC(=O)C1(C)Cl,99 113 314 650 656 667 1041 1057 1060 1135 127...
5,CCc1cc(C=CC(C)=Cc2cc(OC)ccc2C)ccc1C,25 31 80 135 294 322 517 650 694 695 718 781 8...


In [10]:
df.to_csv(output_path)